# Regression Week 4: Ridge Regression (interpretation)

In this notebook, we will run ridge regression multiple times with different L2 penalties to see which one produces the best fit. We will revisit the example of polynomial regression as a means to see the effect of L2 regularization. In particular, we will:
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression
* Use matplotlib to visualize polynomial regressions
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression, this time with L2 penalty
* Use matplotlib to visualize polynomial regressions under L2 regularization
* Choose best L2 penalty using cross-validation.
* Assess the final fit using test data.

We will continue to use the House data from previous notebooks.  (In the next programming assignment for this module, you will implement your own ridge regression learning algorithm using gradient descent.)

# Fire up graphlab create

In [1]:
import graphlab

# Polynomial regression, revisited

We build on the material from Week 3, where we wrote the function to produce an SFrame with columns containing the powers of a given input. Copy and paste the function `polynomial_sframe` from Week 3:

In [2]:
def polynomial_sframe(feature, degree):
     # assume that degree >= 1
    # initialize the SFrame:
    poly_sframe = graphlab.SFrame()
    # and set poly_sframe['power_1'] equal to the passed feature
    poly_sframe['power_1']=feature
    # first check if degree > 1
    if degree > 1:
        # then loop over the remaining degrees:
        # range usually starts at 0 and stops at the endpoint-1. We want it to start at 2 and stop at degree
        for power in range(2, degree+1): 
            # first we'll give the column a name:
            name = 'power_' + str(power)
            # then assign poly_sframe[name] to the appropriate power of feature
            poly_sframe[name] =feature.apply(lambda x: x**power)    
    return poly_sframe

Let's use matplotlib to visualize what a polynomial regression looks like on the house data.

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
sales = graphlab.SFrame('kc_house_data.gl/')

2016-04-17 17:32:31,315 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: C:\Users\Xiuhong\AppData\Local\Temp\graphlab_server_1460928750.log.0


This non-commercial license of GraphLab Create is assigned to xiuhongdu@gmail.com and will expire on March 20, 2017. For commercial licensing options, visit https://dato.com/buy/.


As in Week 3, we will use the sqft_living variable. For plotting purposes (connecting the dots), you'll need to sort by the values of sqft_living. For houses with identical square footage, we break the tie by their prices.

In [6]:
sales = sales.sort(['sqft_living','price'])

Let us revisit the 15th-order polynomial model using the 'sqft_living' input. Generate polynomial features up to degree 15 using `polynomial_sframe()` and fit a model with these features. When fitting the model, use an L2 penalty of `1e-5`:

In [7]:
l2_small_penalty = 1e-5

Note: When we have so many features and so few data points, the solution can become highly numerically unstable, which can sometimes lead to strange unpredictable results.  Thus, rather than using no regularization, we will introduce a tiny amount of regularization (`l2_penalty=1e-5`) to make the solution numerically stable.  (In lecture, we discussed the fact that regularization can also help with numerical stability, and here we are seeing a practical example.)

With the L2 penalty specified above, fit the model and print out the learned weights.

Hint: make sure to add 'price' column to the new SFrame before calling `graphlab.linear_regression.create()`. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set=None` in this call.

In [8]:
poly15_data = polynomial_sframe(sales['sqft_living'], 15)
my_features = poly15_data.column_names()
poly15_data['price'] = sales['price'] # add price to the data since it's the target
model15_p = graphlab.linear_regression.create(poly15_data, target = 'price', features = my_features, l2_penalty=l2_small_penalty, validation_set = None)
model15_p.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 1.042656     | 2662555.738027     | 245656.462165 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,167924.857726,nan
power_1,None,103.090951289,nan
power_2,None,0.13460455096,nan
power_3,None,-0.000129071363752,nan
power_4,None,5.18928955754e-08,nan
power_5,None,-7.77169299595e-12,nan
power_6,None,1.71144842837e-16,nan
power_7,None,4.51177958161e-20,nan
power_8,None,-4.78839816249e-25,nan
power_9,None,-2.33343499941e-28,nan


***QUIZ QUESTION:  What's the learned value for the coefficient of feature `power_1`?***

# Observe overfitting

Recall from Week 3 that the polynomial fit of degree 15 changed wildly whenever the data changed. In particular, when we split the sales data into four subsets and fit the model of degree 15, the result came out to be very different for each subset. The model had a *high variance*. We will see in a moment that ridge regression reduces such variance. But first, we must reproduce the experiment we did in Week 3.

First, split the data into split the sales data into four subsets of roughly equal size and call them `set_1`, `set_2`, `set_3`, and `set_4`. Use `.random_split` function and make sure you set `seed=0`. 

In [9]:
(semi_split1, semi_split2) = sales.random_split(.5,seed=0)
(set_1, set_2) = semi_split1.random_split(0.5, seed=0)
(set_3, set_4) = semi_split2.random_split(0.5, seed=0)

Next, fit a 15th degree polynomial on `set_1`, `set_2`, `set_3`, and `set_4`, using 'sqft_living' to predict prices. Print the weights and make a plot of the resulting model.

Hint: When calling `graphlab.linear_regression.create()`, use the same L2 penalty as before (i.e. `l2_small_penalty`).  Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [10]:
poly15_data_set1 = polynomial_sframe(set_1['sqft_living'], 15)
my_features = poly15_data_set1.column_names()
poly15_data_set1['price'] = set_1['price'] # add price to the data since it's the target
model15_p_set1 = graphlab.linear_regression.create(poly15_data_set1, target = 'price', features = my_features, l2_penalty=l2_small_penalty, validation_set = None)
model15_p_set1.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5404

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.012011     | 2191984.900939     | 248699.117254 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,9306.46397814,835063.417729
power_1,None,585.86581347,3484.52405144
power_2,None,-0.397305884724,5.85325120701
power_3,None,0.000141470894825,0.00510563501632
power_4,None,-1.52945974394e-08,2.39686540035e-06
power_5,None,-3.79756526062e-13,3.43613043735e-10
power_6,None,5.9748184732e-17,nan
power_7,None,1.06888505979e-20,nan
power_8,None,1.59344052349e-25,nan
power_9,None,-6.9283495515e-29,3.72130313789e-25


In [11]:
poly15_data_set2 = polynomial_sframe(set_2['sqft_living'], 15)
my_features = poly15_data_set2.column_names()
poly15_data_set2['price'] = set_2['price'] # add price to the data since it's the target
model15_p_set2 = graphlab.linear_regression.create(poly15_data_set2, target = 'price', features = my_features, l2_penalty=l2_small_penalty, validation_set = None)
model15_p_set2.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5398

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.012009     | 1975178.190682     | 234533.610645 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,-25115.9059869,1675771.52039
power_1,None,783.493802508,9984.85507514
power_2,None,-0.767759300173,25.2592296765
power_3,None,0.000438766361934,0.0358329790553
power_4,None,-1.15169161152e-07,3.17792630727e-05
power_5,None,6.84281148707e-12,1.85840552557e-08
power_6,None,2.5119522464e-15,7.34916239474e-12
power_7,None,-2.06440624344e-19,1.97638603378e-15
power_8,None,-4.59673058828e-23,3.56090427394e-19
power_9,None,-2.71277342492e-29,3.98355514126e-23


In [12]:
poly15_data_set3 = polynomial_sframe(set_3['sqft_living'], 15)
my_features = poly15_data_set3.column_names()
poly15_data_set3['price'] = set_3['price'] # add price to the data since it's the target
model15_p_set3 = graphlab.linear_regression.create(poly15_data_set3, target = 'price', features = my_features, l2_penalty=l2_small_penalty, validation_set = None)
model15_p_set3.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5409

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.013010     | 2283722.683933     | 251097.728054 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,462426.565731,nan
power_1,None,-759.251842854,nan
power_2,None,1.0286700473,nan
power_3,None,-0.000528264527386,nan
power_4,None,1.15422908385e-07,nan
power_5,None,-2.26095948062e-12,nan
power_6,None,-2.08214287571e-15,nan
power_7,None,4.08770475709e-20,nan
power_8,None,2.570791329e-23,nan
power_9,None,1.24311265196e-27,nan


In [13]:
poly15_data_set4 = polynomial_sframe(set_4['sqft_living'], 15)
my_features = poly15_data_set4.column_names()
poly15_data_set4['price'] = set_4['price'] # add price to the data since it's the target
model15_p_set4 = graphlab.linear_regression.create(poly15_data_set4, target = 'price', features = my_features, l2_penalty=l2_small_penalty, validation_set = None)
model15_p_set4.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5402

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.015014     | 2378292.371612     | 244341.293203 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,-170240.034791,1417346.17184
power_1,None,1247.59035088,8978.28059127
power_2,None,-1.2246091264,23.6158213076
power_3,None,0.000555254626787,0.0340561499439
power_4,None,-6.38262361929e-08,2.98955350115e-05
power_5,None,-2.20215996475e-11,1.65791592065e-08
power_6,None,4.81834697594e-15,5.63745618764e-12
power_7,None,4.2146163248e-19,8.27510918329e-16
power_8,None,-7.99880749051e-23,nan
power_9,None,-1.32365907706e-26,nan


The four curves should differ from one another a lot, as should the coefficients you learned.

***QUIZ QUESTION:  For the models learned in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?***  (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Ridge regression comes to rescue

Generally, whenever we see weights change so much in response to change in data, we believe the variance of our estimate to be large. Ridge regression aims to address this issue by penalizing "large" weights. (Weights of `model15` looked quite small, but they are not that small because 'sqft_living' input is in the order of thousands.)

With the argument `l2_penalty=1e5`, fit a 15th-order polynomial model on `set_1`, `set_2`, `set_3`, and `set_4`. Other than the change in the `l2_penalty` parameter, the code should be the same as the experiment above. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [14]:
l2_large_penalty = 1e5
poly15_data_set1 = polynomial_sframe(set_1['sqft_living'], 15)
my_features = poly15_data_set1.column_names()
poly15_data_set1['price'] = set_1['price'] # add price to the data since it's the target
model15_p_set1 = graphlab.linear_regression.create(poly15_data_set1, target = 'price', features = my_features, l2_penalty=l2_large_penalty, validation_set = None)
model15_p_set1.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5404

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.012013     | 5978778.434729     | 374261.720860 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,530317.024516,1256668.1989
power_1,None,2.58738875673,5243.78205387
power_2,None,0.00127414400592,8.80842639712
power_3,None,1.74934226932e-07,0.00768335556793
power_4,None,1.06022119097e-11,3.60698895641e-06
power_5,None,5.42247604482e-16,5.17095558996e-10
power_6,None,2.89563828343e-20,nan
power_7,None,1.65000666351e-24,nan
power_8,None,9.86081528409e-29,nan
power_9,None,6.06589348254e-33,5.60010558786e-25


In [15]:
poly15_data_set2 = polynomial_sframe(set_2['sqft_living'], 15)
my_features = poly15_data_set2.column_names()
poly15_data_set2['price'] = set_2['price'] # add price to the data since it's the target
model15_p_set2 = graphlab.linear_regression.create(poly15_data_set2, target = 'price', features = my_features, l2_penalty=l2_large_penalty, validation_set = None)
model15_p_set2.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5398

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.012011     | 2984894.541944     | 323238.809634 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,519216.897383,2309581.08725
power_1,None,2.04470474182,13761.3225669
power_2,None,0.0011314362684,34.812764407
power_3,None,2.93074277549e-07,0.0493857126218
power_4,None,4.43540598453e-11,4.37988019645e-05
power_5,None,4.80849112204e-15,2.56129084549e-08
power_6,None,4.53091707826e-19,1.0128759361e-11
power_7,None,4.16042910575e-23,2.72389388957e-15
power_8,None,3.90094635128e-27,4.90770792111e-19
power_9,None,3.7773187602e-31,5.49021361344e-23


In [16]:
poly15_data_set3 = polynomial_sframe(set_3['sqft_living'], 15)
my_features = poly15_data_set3.column_names()
poly15_data_set3['price'] = set_3['price'] # add price to the data since it's the target
model15_p_set3 = graphlab.linear_regression.create(poly15_data_set3, target = 'price', features = my_features, l2_penalty=l2_large_penalty, validation_set = None)
model15_p_set3.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5409

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.011516     | 3695342.767093     | 350033.521294 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,522911.518048,nan
power_1,None,2.26890421877,nan
power_2,None,0.00125905041842,nan
power_3,None,2.77552918155e-07,nan
power_4,None,3.2093309779e-11,nan
power_5,None,2.87573572364e-15,nan
power_6,None,2.50076112671e-19,nan
power_7,None,2.24685265906e-23,nan
power_8,None,2.09349983135e-27,nan
power_9,None,2.00435383296e-31,nan


In [17]:
poly15_data_set4 = polynomial_sframe(set_4['sqft_living'], 15)
my_features = poly15_data_set4.column_names()
poly15_data_set4['price'] = set_4['price'] # add price to the data since it's the target
model15_p_set4 = graphlab.linear_regression.create(poly15_data_set4, target = 'price', features = my_features, l2_penalty=l2_large_penalty, validation_set = None)
model15_p_set4.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5402

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.015013     | 3601895.280124     | 323111.582889 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

name,index,value,stderr
(intercept),None,513667.087087,1874267.58319
power_1,None,1.91040938244,11872.6819173
power_2,None,0.00110058029175,31.2290456676
power_3,None,3.12753987879e-07,0.0450351079477
power_4,None,5.50067886825e-11,3.95332017452e-05
power_5,None,7.20467557825e-15,2.19239175825e-08
power_6,None,8.24977249384e-19,7.45484878293e-12
power_7,None,9.06503223498e-23,1.09428234243e-15
power_8,None,9.95683160453e-27,nan
power_9,None,1.10838127982e-30,nan


These curves should vary a lot less, now that you applied a high degree of regularization.

***QUIZ QUESTION:  For the models learned with the high level of regularization in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?*** (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Selecting an L2 penalty via cross-validation

Just like the polynomial degree, the L2 penalty is a "magic" parameter we need to select. We could use the validation set approach as we did in the last module, but that approach has a major disadvantage: it leaves fewer observations available for training. **Cross-validation** seeks to overcome this issue by using all of the training set in a smart way.

We will implement a kind of cross-validation called **k-fold cross-validation**. The method gets its name because it involves dividing the training set into k segments of roughtly equal size. Similar to the validation set method, we measure the validation error with one of the segments designated as the validation set. The major difference is that we repeat the process k times as follows:

Set aside segment 0 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
Set aside segment 1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
...<br>
Set aside segment k-1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set

After this process, we compute the average of the k validation errors, and use it as an estimate of the generalization error. Notice that  all observations are used for both training and validation, as we iterate over segments of data. 

To estimate the generalization error well, it is crucial to shuffle the training data before dividing them into segments. GraphLab Create has a utility function for shuffling a given SFrame. We reserve 10% of the data as the test set and shuffle the remainder. (Make sure to use `seed=1` to get consistent answer.)

In [18]:
(train_valid, test) = sales.random_split(.9, seed=1)
train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=1)

Once the data is shuffled, we divide it into equal segments. Each segment should receive `n/k` elements, where `n` is the number of observations in the training set and `k` is the number of segments. Since the segment 0 starts at index 0 and contains `n/k` elements, it ends at index `(n/k)-1`. The segment 1 starts where the segment 0 left off, at index `(n/k)`. With `n/k` elements, the segment 1 ends at index `(n*2/k)-1`. Continuing in this fashion, we deduce that the segment `i` starts at index `(n*i/k)` and ends at `(n*(i+1)/k)-1`.

With this pattern in mind, we write a short loop that prints the starting and ending indices of each segment, just to make sure you are getting the splits right.

In [19]:
n = len(train_valid_shuffled)
k = 10 # 10-fold cross-validation

for i in xrange(k):
    start = (n*i)/k
    end = (n*(i+1))/k-1
    print i, (start, end)

0 (0, 1938)
1 (1939, 3878)
2 (3879, 5817)
3 (5818, 7757)
4 (7758, 9697)
5 (9698, 11636)
6 (11637, 13576)
7 (13577, 15515)
8 (15516, 17455)
9 (17456, 19395)


Let us familiarize ourselves with array slicing with SFrame. To extract a continuous slice from an SFrame, use colon in square brackets. For instance, the following cell extracts rows 0 to 9 of `train_valid_shuffled`. Notice that the first index (0) is included in the slice but the last index (10) is omitted.

In [20]:
train_valid_shuffled[0:10] # rows 0 to 9

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
2780400035,2014-05-05 00:00:00+00:00,665000.0,4.0,2.5,2800.0,5900,1,0
1703050500,2015-03-21 00:00:00+00:00,645000.0,3.0,2.5,2490.0,5978,2,0
5700002325,2014-06-05 00:00:00+00:00,640000.0,3.0,1.75,2340.0,4206,1,0
0475000510,2014-11-18 00:00:00+00:00,594000.0,3.0,1.0,1320.0,5000,1,0
0844001052,2015-01-28 00:00:00+00:00,365000.0,4.0,2.5,1904.0,8200,2,0
2781280290,2015-04-27 00:00:00+00:00,305000.0,3.0,2.5,1610.0,3516,2,0
2214800630,2014-11-05 00:00:00+00:00,239950.0,3.0,2.25,1560.0,8280,2,0
2114700540,2014-10-21 00:00:00+00:00,366000.0,3.0,2.5,1320.0,4320,1,0
2596400050,2014-07-30 00:00:00+00:00,375000.0,3.0,1.0,1960.0,7955,1,0
4140900050,2015-01-26 00:00:00+00:00,440000.0,4.0,1.75,2180.0,10200,1,0


Now let us extract individual segments with array slicing. Consider the scenario where we group the houses in the `train_valid_shuffled` dataframe into k=10 segments of roughly equal size, with starting and ending indices computed as above.
Extract the fourth segment (segment 3) and assign it to a variable called `validation4`.

In [21]:
validation4 = train_valid_shuffled[5818:7758]

To verify that we have the right elements extracted, run the following cell, which computes the average price of the fourth segment. When rounded to nearest whole number, the average should be $536,234.

In [22]:
print int(round(validation4['price'].mean(), 0))

536234


After designating one of the k segments as the validation set, we train a model using the rest of the data. To choose the remainder, we slice (0:start) and (end+1:n) of the data and paste them together. SFrame has `append()` method that pastes together two disjoint sets of rows originating from a common dataset. For instance, the following cell pastes together the first and last two rows of the `train_valid_shuffled` dataframe.

In [23]:
n = len(train_valid_shuffled)

first_two = train_valid_shuffled[0:2]
last_two = train_valid_shuffled[n-2:n]
print first_two.append(last_two)

+------------+---------------------------+-----------+----------+-----------+
|     id     |            date           |   price   | bedrooms | bathrooms |
+------------+---------------------------+-----------+----------+-----------+
| 2780400035 | 2014-05-05 00:00:00+00:00 |  665000.0 |   4.0    |    2.5    |
| 1703050500 | 2015-03-21 00:00:00+00:00 |  645000.0 |   3.0    |    2.5    |
| 4139480190 | 2014-09-16 00:00:00+00:00 | 1153000.0 |   3.0    |    3.25   |
| 7237300290 | 2015-03-26 00:00:00+00:00 |  338000.0 |   5.0    |    2.5    |
+------------+---------------------------+-----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    2800.0   |   5900   |   1    |     0      |  0   |     3     |   8   |    1660    |
|    2490.0   |   59

Extract the remainder of the data after *excluding* fourth segment (segment 3) and assign the subset to `train4`.

In [24]:
n = len(train_valid_shuffled)
i=3
start = (n*i)/k
end = (n*(i+1))/k-1
first_three = train_valid_shuffled[0:start]
last_three = train_valid_shuffled[end+1:n]
train4 = first_three.append(last_three)

To verify that we have the right elements extracted, run the following cell, which computes the average price of the data with fourth segment excluded. When rounded to nearest whole number, the average should be $539,450.

In [25]:
print int(round(train4['price'].mean(), 0))

539450


Now we are ready to implement k-fold cross-validation. Write a function that computes k validation errors by designating each of the k segments as the validation set. It accepts as parameters (i) `k`, (ii) `l2_penalty`, (iii) dataframe, (iv) name of output column (e.g. `price`) and (v) list of feature names. The function returns the average validation error using k segments as validation sets.

* For each i in [0, 1, ..., k-1]:
  * Compute starting and ending indices of segment i and call 'start' and 'end'
  * Form validation set by taking a slice (start:end+1) from the data.
  * Form training set by appending slice (end+1:n) to the end of slice (0:start).
  * Train a linear model using training set just formed, with a given l2_penalty
  * Compute validation error using validation set just formed

In [26]:
import numpy as np
def k_fold_cross_validation(k, l2_penalty, data, output_name, features_list):
    validation_error = np.zeros(k)
    for i in range(k):
        start = i*n/k
        end = (i+1)*n/k-1
        validation_set = data[start:end+1]
        first_set = data[0:start]
        last_set = data[end+1:n]
        train_set = first_set.append(last_set)
        poly15_data = train_set[features_list]
        my_features = features_list
        poly15_data[output_name] = train_set[output_name] # add price to the data since it's the target
        model15_p = graphlab.linear_regression.create(poly15_data, target = output_name, features = my_features, l2_penalty=l2_penalty, validation_set = None)
        validation_set['prediction'] = model15_p.predict(validation_set)
        validation_error[i] = np.dot(validation_set['prediction']-validation_set[output_name],validation_set['prediction']-validation_set[output_name])
    average_validation_error = validation_error.mean()
    return average_validation_error

Once we have a function to compute the average validation error for a model, we can write a loop to find the model that minimizes the average validation error. Write a loop that does the following:
* We will again be aiming to fit a 15th-order polynomial model using the `sqft_living` input
* For `l2_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, you can use this Numpy function: `np.logspace(1, 7, num=13)`.)
    * Run 10-fold cross-validation with `l2_penalty`
* Report which L2 penalty produced the lowest average validation error.

Note: since the degree of the polynomial is now fixed to 15, to make things faster, you should generate polynomial features in advance and re-use them throughout the loop. Make sure to use `train_valid_shuffled` when generating polynomial features!

In [27]:
poly15_data_new = polynomial_sframe(train_valid_shuffled['sqft_living'], 15)
features_list = poly15_data_new.column_names()
poly15_data_new['price'] = train_valid_shuffled['price']
average_error = np.zeros(13)
i=0
k=10
data = poly15_data_new
output_name = 'price'

for l2_penalty in np.logspace(1, 7, num=13):
    average_error[i]=k_fold_cross_validation(k, l2_penalty, data, output_name, features_list)
    i = i+1
    #print i, average_error[i-1]

Linear regression:

--------------------------------------------------------

Number of examples          : 17457

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.036026     | 6492747.587172     | 366041.873855 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.


2.64889015378e+14


***QUIZ QUESTIONS:  What is the best value for the L2 penalty according to 10-fold validation?***

You may find it useful to plot the k-fold cross-validation errors you have obtained to better understand the behavior of the method.  

In [28]:
# Plot the l2_penalty values in the x axis and the cross-validation error in the y axis.
# Using plt.xscale('log') will make your plot more intuitive.
i =0
for l2_penalty in np.logspace(1, 7, num=13):
    print l2_penalty
    print i, average_error[i]
    i = i+1



10.0
0 4.91826427769e+14
31.6227766017
1 2.87504229919e+14
100.0
2 1.60908965822e+14
316.227766017
3 1.22090967326e+14
1000.0
4 1.21192264451e+14
3162.27766017
5 1.2395000929e+14
10000.0
6 1.36837175248e+14
31622.7766017
7 1.71728094842e+14
100000.0
8 2.2936143126e+14
316227.766017
9 2.52940568729e+14
1000000.0
10 2.58682548441e+14
3162277.66017
11 2.62819399742e+14
10000000.0
12 2.64889015378e+14


Once you found the best value for the L2 penalty using cross-validation, it is important to retrain a final model on all of the training data using this value of `l2_penalty`.  This way, your final model will be trained on the entire dataset.

In [29]:
l2_penalty_opt = 1000
test_data = polynomial_sframe(test['sqft_living'], 15)
#features_list = test_data.column_names()
test_data['price'] = test['price']
model15_p_opt = graphlab.linear_regression.create(test_data, target = output_name, features = features_list, l2_penalty=l2_penalty_opt, validation_set = None)
test_data['prediction'] = model15_p_opt.predict(test_data)
test_error = np.dot(test_data['prediction']-test_data[output_name],test_data['prediction']-test_data[output_name])
print test_error


Linear regression:

--------------------------------------------------------

Number of examples          : 2217

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.005006     | 1563748.163823     | 244004.602702 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

1.31996291692e+14


***QUIZ QUESTION: Using the best L2 penalty found above, train a model using all training data. What is the RSS on the TEST data of the model you learn with this L2 penalty? ***